In [1]:
import typing as t
import operator as op
import subprocess as sp
import glob
import os
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from itertools import chain

import tqdm
import pandas as pd
from fn import F
from Bio import SearchIO, SeqIO

pd.set_option('display.max_rows', 1000)
%matplotlib inline

In [1]:
#!pip3 install tqdm

     |████████████████████████████████| 59 kB 1.6 MB/s eta 0:00:011


In [3]:
#!pip3 install pandas

     |████████████████████████████████| 10.1 MB 3.9 MB/s eta 0:00:01    |▏                               | 40 kB 3.0 MB/s eta 0:00:04
     |████████████████████████████████| 20.1 MB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 7.0 MB/s eta 0:00:01


In [5]:
#!pip3 install fn

  Created wheel for fn: filename=fn-0.4.3-py3-none-any.whl size=28473 sha256=e91ac167f87d0f446ff280a5ec8026e21a1edad60ee031f81af77d91c6b7e903
  Stored in directory: /home/is6/.cache/pip/wheels/5c/a0/65/41f733d04386c57826c2cf9e1927d716e90a46689712c9af6b
Successfully built fn


In [10]:
#!pip3 install biopython

     |████████████████████████████████| 2.3 MB 4.5 MB/s eta 0:00:01


In [12]:
#!pip3 install matplotlib

     |████████████████████████████████| 12.4 MB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 4.3 MB/s eta 0:00:011
     |████████████████████████████████| 67 kB 2.3 MB/s eta 0:00:011


In [ ]:
# all modules have been installed

Parse hmmscan output and extract domain annotations

In [2]:
scan_files = [
    'hmmscan/sp.txt',
    'hmmscan/chz.txt'
]
samples = ['sp', 'chz']

scans = [
    list(SearchIO.parse(fname, 'hmmer3-text')) for fname in scan_files
]


In [3]:
hmmscan_annotation_keys = [
    'query', 'model', 'description', 'bitscore', 
    'evalue', 'domains_expected', 'domains_observed'
]

def extract_query_annotations(query) -> pd.DataFrame:
    query_id = query.id
    hits = query.hits
    hit_ids = [hit.id for hit in hits]
    hit_descriptions = [hit.description for hit in hits]
    hit_scores = [hit.bitscore for hit in hits]
    hit_evalues = [hit.evalue for hit in hits]
    domains_expected = [hit.domain_exp_num for hit in hits]
    domains_observed = [hit.domain_obs_num for hit in hits]
    records = [
        [query_id, hit_id, hit_descr, hit_score, hit_eval, dom_exp, dom_obs]
        for hit_id, hit_descr, hit_score, hit_eval, dom_exp, dom_obs, in 
        zip(hit_ids, hit_descriptions, hit_scores, hit_evalues,
            domains_expected, domains_observed)
    ]
    return pd.DataFrame(records, columns=hmmscan_annotation_keys)


def extract_hmmscan_annotations(sample_id, sample_queries) -> pd.DataFrame:
    empty = pd.DataFrame(
        data=[['NA', 'NA', 'NA', 0, 99999.0, 0.0, 0.0]], 
        columns=hmmscan_annotation_keys
    )
    hmmscan = (
        F(map, extract_query_annotations)
        >> list
        >> (lambda dfs: pd.concat(dfs) if dfs else empty)
        >> (lambda df: df.reset_index(drop=True))
    )(sample_queries)
    return pd.concat([
        pd.Series(sample_id, index=hmmscan.index, name='sample'),
        hmmscan
    ], axis=1)


In [5]:
scan_annotations = [
    extract_hmmscan_annotations(id_, scan) for id_, scan in zip(samples, scans)
]


In [6]:
scan_annotations_concat = (
    pd.concat(scan_annotations)
    .reset_index(drop=True)
    .sort_values(['sample', 'description'])
)
scan_annotations_concat.head()


,sample,query,model,description,bitscore,evalue,domains_expected,domains_observed
122,chz,NFBJIBDF_18870,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,142.6,1.500000e-44,1.3,1
134,chz,NFBJIBDF_33696,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,139.1,1.900000e-43,1.8,2
138,chz,NFBJIBDF_36526,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,99.0,4.000000e-31,1.4,1
144,chz,NFBJIBDF_42607,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,131.8,3.300000e-41,1.2,1
155,chz,NFBJIBDF_49966,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,98.1,7.500000e-31,1.0,1


In [58]:
#scan_annotations_concat.tail()

,sample,query,model,description,bitscore,evalue,domains_expected,domains_observed
294,chz,NFBJIBDF_95712,PF00331.20,Glycosyl hydrolase family 10,170.0,1.500000e-52,1.0,1
295,chz,NFBJIBDF_96180,PF01915.22,Glycosyl hydrolase family 3 C-terminal domain,153.9,1.100000e-47,1.2,1
296,chz,NFBJIBDF_97416,PF01670.16,Glycosyl hydrolase family 12,93.6,3.600000e-29,1.0,1
297,chz,NFBJIBDF_99251,PF12215.8,"beta-glucosidase 2, glycosyl-hydrolase family ...",70.1,5.000000e-22,1.0,1
298,chz,NFBJIBDF_99268,PF00722.21,Glycosyl hydrolases family 16,63.0,5.200000e-20,1.4,1


In [15]:
# ambiguity check: several domain profiles matching the same query?
scan_annotations_concat.groupby('query').apply(len).max()

3

In [16]:
# ambiguity check: several domain profiles matching the same query? 
scan_annotations_concat.groupby('query').apply(len) >1

query
LDFJCBPN_00383    False
LDFJCBPN_02521    False
LDFJCBPN_02523    False
LDFJCBPN_03057    False
LDFJCBPN_04133    False
LDFJCBPN_04134    False
LDFJCBPN_04141    False
LDFJCBPN_04191    False
LDFJCBPN_05098    False
LDFJCBPN_05318    False
LDFJCBPN_06314    False
LDFJCBPN_06822    False
LDFJCBPN_06823    False
LDFJCBPN_06824    False
LDFJCBPN_07844    False
LDFJCBPN_07848    False
LDFJCBPN_07851    False
LDFJCBPN_08501    False
LDFJCBPN_11484    False
LDFJCBPN_11973    False
LDFJCBPN_11974    False
LDFJCBPN_11975    False
LDFJCBPN_12341    False
LDFJCBPN_12342    False
LDFJCBPN_13622    False
LDFJCBPN_15911    False
LDFJCBPN_15912    False
LDFJCBPN_16036    False
LDFJCBPN_16159    False
LDFJCBPN_17218    False
LDFJCBPN_18009    False
LDFJCBPN_18946    False
LDFJCBPN_21748    False
LDFJCBPN_21749    False
LDFJCBPN_22852    False
LDFJCBPN_22853    False
LDFJCBPN_23110    False
LDFJCBPN_24967    False
LDFJCBPN_28973    False
LDFJCBPN_29859    False
LDFJCBPN_30665    False
LDFJCBPN_3

In [17]:
scan_annotations_concat

,sample,query,model,description,bitscore,evalue,domains_expected,domains_observed
122,chz,NFBJIBDF_18870,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,142.6,1.500000e-44,1.3,1
134,chz,NFBJIBDF_33696,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,139.1,1.900000e-43,1.8,2
138,chz,NFBJIBDF_36526,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,99.0,4.000000e-31,1.4,1
144,chz,NFBJIBDF_42607,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,131.8,3.300000e-41,1.2,1
155,chz,NFBJIBDF_49966,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,98.1,7.500000e-31,1.0,1
158,chz,NFBJIBDF_51960,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,147.9,3.800000e-46,1.0,1
159,chz,NFBJIBDF_51963,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,107.3,1.200000e-33,1.0,1
162,chz,NFBJIBDF_56144,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,96.5,2.300000e-30,1.2,1
87,chz,NFBJIBDF_06277,PF00150.18,Cellulase (glycosyl hydrolase family 5),103.5,1.800000e-32,1.1,1
94,chz,NFBJIBDF_08293,PF00150.18,Cellulase (glycosyl hydrolase family 5),82.8,3.500000e-26,1.1,1


In [48]:
# there are some duplicates

In [8]:
len(scan_annotations_concat)

167

In [9]:
# sorting by first name 
#scan_annotations_concat.sort_values("query", inplace = True) 
  
# dropping ALL duplicte values 
#scan_annotations_concat.drop_duplicates(subset ="query", 
#                     keep = False, inplace = True) 

In [ ]:
# we have removed the duplicates

In [10]:
os.makedirs('glyco', exist_ok=True)


In [11]:
# export domain annotations
scan_annotations_concat.to_csv('glyco/annotations.tsv', sep='\t', index=False)


Extract nucleotide (or protein) sequences with positive matches

In [12]:
scan_annotations_concat['description'].value_counts()


Glycosyl hydrolase family 3 N terminal domain      37
Glycosyl hydrolase family 3 C-terminal domain      26
Glycosyl hydrolases family 43                      19
Cellulase (glycosyl hydrolase family 5)            14
Alpha-L-arabinofuranosidase C-terminal domain      10
Glycosyl hydrolases family 6                       10
Glycosyl hydrolase family 10                        9
Glycosyl hydrolases family 16                       8
Glycosyl hydrolases family 8                        7
Glycosyl hydrolase family 67 middle domain          5
Glycosyl hydrolase family 115                       5
Glycosyl hydrolase family 9                         4
Glycosyl hydrolases family 11                       3
Glycosyl hydrolase family 67 C-terminus             3
Glycosyl hydrolase family 67 N-terminus             2
Glycosyl hydrolases family 39                       2
Gylcosyl hydrolase family 115 C-terminal domain     1
Glycosyl hydrolase family 26                        1
Glycoside hydrolase family 4

In [13]:
selected_queries = set(scan_annotations_concat['query'])
proteins = [
    SeqIO.parse('annotation/sp/sp_assembly.fasta.ffn', 'fasta'),
    SeqIO.parse('annotation/chz/chz_assembly.fasta.ffn', 'fasta')
]
for samp, prots in zip(samples, proteins):
    SeqIO.write((seq for seq in prots if seq.id in selected_queries), f'glyco/{samp}.faa', 'fasta')